# Structure Server - Comprehensive Test Suite

This notebook tests all MCP tools in `structure_server.py`:

1. **fetch_molecules** - Fetch structures from PDB/AlphaFold/PDB-REDO
2. **inspect_molecules** - Inspect structure files to analyze chains and molecules
3. **split_molecules** - Split multi-chain structures into individual chains
4. **clean_protein** - Clean protein structures for MD simulation
5. **clean_ligand** - Clean and prepare ligands using SMILES template matching
6. **run_antechamber_robust** - GAFF2 parameterization with AM1-BCC charges
7. **prepare_complex** - Complete workflow (split + clean + parameterize)

Each tool is tested for:
- Normal operation
- Edge cases
- Error handling (LLM-friendly error responses)
- Boltz-2 predicted structures (computational models)
- Ligand preparation and force field generation


In [16]:
# Setup
import sys
sys.path.insert(0, '..')

from pathlib import Path
import json
import importlib
import asyncio

# For running async functions in notebook
import nest_asyncio
nest_asyncio.apply()

print("Setup complete")


Setup complete


In [17]:
# Check dependencies
print("Checking dependencies...\n")

deps = {
    "gemmi": "Structure parsing (mmCIF/PDB)",
    "pdbfixer": "Protein structure cleaning",
    "openmm": "Molecular simulation",
    "httpx": "Async HTTP client",
    "rdkit": "Ligand processing and charge estimation"
}

for module, desc in deps.items():
    try:
        __import__(module)
        print(f"✓ {module}: {desc}")
    except ImportError:
        print(f"✗ {module}: {desc} (NOT INSTALLED)")

# Check external tools
print("\nChecking external tools...")
from common.base import BaseToolWrapper

tools = {
    "pdb4amber": "Amber naming conventions",
    "antechamber": "GAFF2 parameterization",
    "parmchk2": "Missing parameter generation",
    "obabel": "Format conversion"
}

for tool, desc in tools.items():
    wrapper = BaseToolWrapper(tool)
    print(f"{'✓' if wrapper.is_available() else '✗'} {tool} ({desc})")


Checking dependencies...

✓ gemmi: Structure parsing (mmCIF/PDB)
✓ pdbfixer: Protein structure cleaning
✓ openmm: Molecular simulation
✓ httpx: Async HTTP client
✓ rdkit: Ligand processing and charge estimation

Checking external tools...
✓ pdb4amber (Amber naming conventions)
✓ antechamber (GAFF2 parameterization)
✓ parmchk2 (Missing parameter generation)
✓ obabel (Format conversion)


In [18]:
# Import and reload the structure server module
import servers.structure_server as structure_module
importlib.reload(structure_module)

# Import tools directly
from servers.structure_server import (
    fetch_molecules,
    inspect_molecules,
    split_molecules,
    clean_protein,
    clean_ligand,
    run_antechamber_robust,
    prepare_complex
)

print("Structure server tools imported successfully")


[12/16/25 17:53:42] WARNING  FASPR not found in PATH                                                    ]8;id=873963;file:///Users/yasu/tmp/mcp-md/notebooks/../common/base.py\base.py]8;;\:]8;id=859915;file:///Users/yasu/tmp/mcp-md/notebooks/../common/base.py#103\103]8;;\

Structure server tools imported successfully


In [19]:
# Helper function to display results nicely
def show_result(result: dict, title: str = "Result"):
    """Display result dictionary with formatting"""
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")
    
    # Check success status
    if result.get('success'):
        print("\n✓ SUCCESS")
    else:
        print("\n✗ FAILED")
    
    # Show errors if any
    if result.get('errors'):
        print("\nErrors:")
        for err in result['errors']:
            print(f"  - {err}")
    
    # Show warnings if any
    if result.get('warnings'):
        print("\nWarnings:")
        for warn in result['warnings']:
            print(f"  - {warn}")
    
    # Show key fields
    skip_keys = {'success', 'errors', 'warnings', 'operations'}
    print("\nDetails:")
    for k, v in result.items():
        if k not in skip_keys:
            if isinstance(v, (dict, list)) and len(str(v)) > 100:
                print(f"  {k}: [complex data, {len(v) if isinstance(v, list) else 'dict'}]")
            else:
                print(f"  {k}: {v}")
    
    # Show operations if present
    if result.get('operations'):
        print("\nOperations:")
        for op in result['operations']:
            status_icon = "✓" if op.get('status') in ['success', 'detected', 'added', 'replaced'] else "○"
            print(f"  {status_icon} {op.get('step')}: {op.get('status')} - {op.get('details', '')[:60]}")

print("Helper function defined")


Helper function defined


---
## Test 1: fetch_molecules

Test fetching structures from different sources.


In [20]:
# Test 1.1: Fetch from PDB (small protein: 1CRN - crambin)
print("Test 1.1: Fetch 1CRN from PDB")

result = asyncio.run(fetch_molecules("1CRN", source="pdb"))
show_result(result, "Fetch 1CRN from PDB")

# Verify file exists
if result['success'] and result['file_path']:
    print(f"\nFile size: {Path(result['file_path']).stat().st_size} bytes")


2025-12-16 17:53:42,503 - servers.structure_server - INFO - Fetching 1CRN from pdb


Test 1.1: Fetch 1CRN from PDB


                    INFO     Fetching 1CRN from pdb                                         ]8;id=768285;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=734684;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

[12/16/25 17:53:54] INFO     HTTP Request: GET https://files.rcsb.org/download/1CRN.pdb "HTTP/1.1   ]8;id=761193;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=318947;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-16 17:53:54,949 - servers.structure_server - INFO - Downloaded 1CRN to output/1CRN.pdb


                    INFO     Downloaded 1CRN to output/1CRN.pdb                             ]8;id=387199;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=719333;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#781\781]8;;\

2025-12-16 17:53:54,955 - servers.structure_server - INFO - Successfully fetched 1CRN: 327 atoms, chains: ['A']


                    INFO     Successfully fetched 1CRN: 327 atoms, chains: ['A']            ]8;id=560338;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=53837;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#817\817]8;;\


 Fetch 1CRN from PDB

✓ SUCCESS

Details:
  pdb_id: 1CRN
  source: pdb
  file_path: output/1CRN.pdb
  file_format: pdb
  num_atoms: 327
  chains: ['A']

File size: 49491 bytes


In [21]:
# Test 1.2: Fetch non-existent PDB ID (error handling)
print("Test 1.2: Fetch non-existent PDB ID")

result = asyncio.run(fetch_molecules("XXXX", source="pdb"))
show_result(result, "Fetch Invalid PDB ID")

# Check that error handling is LLM-friendly
assert not result['success'], "Should fail for invalid PDB ID"
assert len(result['errors']) > 0, "Should have error messages"
print("\n✓ Error handling works correctly")


2025-12-16 17:53:54,964 - servers.structure_server - INFO - Fetching XXXX from pdb


Test 1.2: Fetch non-existent PDB ID


                    INFO     Fetching XXXX from pdb                                         ]8;id=842586;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=441179;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

[12/16/25 17:54:01] INFO     HTTP Request: GET https://files.rcsb.org/download/XXXX.pdb "HTTP/1.1   ]8;id=40690;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=795384;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             404 Not Found"                                                                        

                    INFO     HTTP Request: GET https://files.rcsb.org/download/XXXX.cif "HTTP/1.1   ]8;id=652628;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=197883;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             404 Not Found"                                                                        


 Fetch Invalid PDB ID

✗ FAILED

Errors:
  - Structure not found: XXXX (HTTP 404)
  - Hint: Verify the PDB ID is correct. Try searching at https://www.rcsb.org/

Warnings:
  - PDB format not available, falling back to mmCIF

Details:
  pdb_id: XXXX
  source: pdb
  file_path: None
  file_format: None
  num_atoms: 0
  chains: []

✓ Error handling works correctly


---
## Test 2: inspect_molecules

Test inspecting structure files to analyze chains and molecular composition.


In [22]:
# Test 2.1: Inspect 1AKE (homodimer with ligand)
print("Test 2.1: Inspect 1AKE structure")

# First fetch 1AKE
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    result = inspect_molecules(fetch_result['file_path'])
    show_result(result, "Inspect 1AKE")
    
    # Show detailed chain information
    if result['success']:
        print("\n--- Header Information ---")
        for k, v in result.get('header', {}).items():
            print(f"  {k}: {v}")
        
        print("\n--- Entities (from header) ---")
        for entity in result.get('entities', []):
            print(f"  Entity {entity['entity_id']}: {entity.get('name') or '(no name)'}")
            print(f"    Type: {entity['entity_type']}, Polymer: {entity.get('polymer_type')}")
            print(f"    Chains: {entity['chain_ids']}")
        
        print("\n--- Chain Summary ---")
        summary = result.get('summary', {})
        print(f"  Proteins: {summary.get('num_protein_chains', 0)} chains {summary.get('protein_chain_ids', [])}")
        print(f"  Ligands: {summary.get('num_ligand_chains', 0)} chains {summary.get('ligand_chain_ids', [])}")
        print(f"  Waters: {summary.get('num_water_chains', 0)} chains {summary.get('water_chain_ids', [])}")
        print(f"  Ions: {summary.get('num_ion_chains', 0)} chains {summary.get('ion_chain_ids', [])}")
        
        print("\n--- Chains Detail ---")
        for chain in result.get('chains', []):
            print(f"  Chain {chain['chain_id']} ({chain['author_chain']}): {chain['chain_type']}")
            print(f"    Entity: {chain.get('entity_name') or chain.get('entity_id') or 'N/A'}")
            print(f"    Residues: {chain['num_residues']}, Atoms: {chain['num_atoms']}")
            if chain.get('sequence'):
                seq = chain['sequence']
                print(f"    Sequence: {seq[:50]}{'...' if len(seq) > 50 else ''}")
else:
    print("Failed to fetch 1AKE for inspect test")


2025-12-16 17:54:01,686 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 2.1: Inspect 1AKE structure


                    INFO     Fetching 1AKE from pdb                                         ]8;id=714181;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=796162;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

[12/16/25 17:54:02] INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.pdb "HTTP/1.1   ]8;id=729147;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=708674;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-16 17:54:02,421 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.pdb


...

2025-12-16 17:54:02,483 - servers.structure_server - INFO - Successfully inspected structure: 6 chains found


                    INFO     Successfully inspected structure: 6 chains found              ]8;id=622926;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=984820;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1158\1158]8;;\

2025-12-16 17:54:02,484 - servers.structure_server - INFO -   Proteins: 2, Ligands: 2, Waters: 2, Ions: 0


                    INFO       Proteins: 2, Ligands: 2, Waters: 2, Ions: 0                 ]8;id=992632;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=803587;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1159\1159]8;;\


 Inspect 1AKE

✓ SUCCESS

Details:
  source_file: output/1AKE.pdb
  file_format: pdb
  header: [complex data, dict]
  entities: [complex data, 3]
  num_models: 1
  chains: [complex data, 6]
  summary: [complex data, dict]

--- Header Information ---
  pdb_id: 1AKE
  title: STRUCTURE OF THE COMPLEX BETWEEN ADENYLATE KINASE FROM ESCHERICHIA COLI AND THE INHIBITOR AP5A REFINED AT 1.9 ANGSTROMS RESOLUTION: A MODEL FOR A CATALYTIC TRANSITION STATE
  resolution: 2.0
  spacegroup: P 21 2 21
  experiment_method: X-RAY DIFFRACTION

--- Entities (from header) ---
  Entity A: (no name)
    Type: polymer, Polymer: PeptideL
    Chains: ['Axp', 'Bxp']
  Entity AP5!: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['Ax1', 'Bx1']
  Entity water: (no name)
    Type: water, Polymer: None
    Chains: ['Axw', 'Bxw']

--- Chain Summary ---
  Proteins: 2 chains ['Axp', 'Bxp']
  Ligands: 2 chains ['Ax1', 'Bx1']
  Waters: 2 chains ['Axw', 'Bxw']
  Ions: 0 chains []

--- Chains Detail ---
  Chain Ax

In [23]:
# Test 2.2: Inspect Boltz-2 predicted structure (computational model)
print("Test 2.2: Inspect Boltz-2 predicted structure")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
result = inspect_molecules(boltz_cif)
show_result(result, "Inspect Boltz-2 Prediction")

# Show detailed information for AI-generated structure
if result['success']:
    print("\n--- Header Information ---")
    for k, v in result.get('header', {}).items():
        print(f"  {k}: {v}")
    
    print("\n--- Entities ---")
    for entity in result.get('entities', []):
        print(f"  Entity {entity['entity_id']}: {entity.get('name') or '(no name)'}")
        print(f"    Type: {entity['entity_type']}, Polymer: {entity.get('polymer_type')}")
        print(f"    Chains: {entity['chain_ids']}")
    
    print("\n--- Chains Summary ---")
    summary = result.get('summary', {})
    print(f"  Proteins: {summary.get('num_protein_chains', 0)} chains")
    print(f"  Ligands: {summary.get('num_ligand_chains', 0)} chains")
    
    print("\n--- Chain Details ---")
    for chain in result.get('chains', []):
        print(f"  Chain {chain['chain_id']}: {chain['chain_type']}")
        print(f"    Residues: {chain['num_residues']}, Atoms: {chain['num_atoms']}")
        print(f"    Residue types: {chain['residue_names']['unique_residues']}")


2025-12-16 17:54:02,490 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 2.2: Inspect Boltz-2 predicted structure


                    INFO     Inspecting molecules in:                                       ]8;id=850614;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=332175;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:54:02,491 - servers.structure_server - INFO - Reading structure with gemmi (.cif)...


                    INFO     Reading structure with gemmi (.cif)...                         ]8;id=192400;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=404355;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#958\958]8;;\

2025-12-16 17:54:02,515 - servers.structure_server - INFO - Successfully inspected structure: 6 chains found


                    INFO     Successfully inspected structure: 6 chains found              ]8;id=448829;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=144459;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1158\1158]8;;\

2025-12-16 17:54:02,516 - servers.structure_server - INFO -   Proteins: 2, Ligands: 4, Waters: 0, Ions: 0


                    INFO       Proteins: 2, Ligands: 4, Waters: 0, Ions: 0                 ]8;id=933487;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=26698;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1159\1159]8;;\


 Inspect Boltz-2 Prediction

✓ SUCCESS

Details:
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  file_format: cif
  header: {'pdb_id': 'model'}
  entities: [complex data, 3]
  num_models: 1
  chains: [complex data, 6]
  summary: [complex data, dict]

--- Header Information ---
  pdb_id: model

--- Entities ---
  Entity 1: (no name)
    Type: polymer, Polymer: PeptideL
    Chains: ['A', 'B']
  Entity 2: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['C', 'D']
  Entity 3: (no name)
    Type: nonpolymer, Polymer: None
    Chains: ['E', 'F']

--- Chains Summary ---
  Proteins: 2 chains
  Ligands: 4 chains

--- Chain Details ---
  Chain A: protein
    Residues: 384, Atoms: 2961
    Residue types: ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE']
  Chain B: protein
    Residues: 384, Atoms: 2961
    Residue types: ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE']
  Chain C: ligand
    Residues: 1, Atoms: 26
  

---
## Test 3: split_molecules

Test splitting multi-chain structures into individual chain files.
The `split_molecules` function uses `inspect_molecules` internally.


In [24]:
# Test 3.1: Split 1AKE (homodimer with ligand)
print("Test 3.1: Split 1AKE structure")

# First fetch 1AKE (if not already available)
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    result = split_molecules(fetch_result['file_path'])
    show_result(result, "Split 1AKE")
    
    # Show chain files
    if result['success']:
        print("\nProtein files:")
        for f in result['protein_files']:
            print(f"  - {f}")
        print("\nLigand files:")
        for f in result['ligand_files']:
            print(f"  - {f}")
        if result['ion_files']:
            print("\nIon files:")
            for f in result['ion_files']:
                print(f"  - {f}")
        
        # Show chain mapping
        print("\nChain to file mapping:")
        for info in result.get('chain_file_info', []):
            print(f"  Chain {info['chain_id']} ({info['chain_type']}): {info['file']}")
else:
    print("Failed to fetch 1AKE for split test")


2025-12-16 17:54:02,523 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 3.1: Split 1AKE structure


                    INFO     Fetching 1AKE from pdb                                         ]8;id=931026;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=655931;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.pdb "HTTP/1.1   ]8;id=352692;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=638270;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-16 17:54:03,004 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.pdb


...

2025-12-16 17:54:03,088 - servers.structure_server - INFO - Wrote ligand: output/7f557747/ligand_2.pdb


                    INFO     Wrote ligand: output/7f557747/ligand_2.pdb                    ]8;id=694748;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=732679;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1454\1454]8;;\

2025-12-16 17:54:03,089 - servers.structure_server - INFO - Successfully split structure: 2 protein, 2 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 2 protein, 2 ligand, 0 ion, 0   ]8;id=163440;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=320445;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1481\1481]8;;\
                             water files                                                                           


 Split 1AKE

✓ SUCCESS

Details:
  job_id: 7f557747
  output_dir: output/7f557747
  source_file: output/1AKE.pdb
  file_format: pdb
  protein_files: ['output/7f557747/protein_1.pdb', 'output/7f557747/protein_2.pdb']
  ligand_files: ['output/7f557747/ligand_1.pdb', 'output/7f557747/ligand_2.pdb']
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 4]
  include_types: ['protein', 'ligand', 'ion']

Protein files:
  - output/7f557747/protein_1.pdb
  - output/7f557747/protein_2.pdb

Ligand files:
  - output/7f557747/ligand_1.pdb
  - output/7f557747/ligand_2.pdb

Chain to file mapping:
  Chain Axp (protein): output/7f557747/protein_1.pdb
  Chain Bxp (protein): output/7f557747/protein_2.pdb
  Chain Ax1 (ligand): output/7f557747/ligand_1.pdb
  Chain Bx1 (ligand): output/7f557747/ligand_2.pdb


In [25]:
# Test 3.2: Split with chain selection
print("Test 3.2: Split 1AKE - select only chain A")

if fetch_result['success']:
    result = split_molecules(
        fetch_result['file_path'],
        select_chains=['A']
    )
    show_result(result, "Split 1AKE (Chain A only)")
    
    if result['success']:
        print(f"\nExtracted {len(result['protein_files'])} protein chain(s)")
        print(f"Output directory: {result['output_dir']}")
else:
    print("Skipped - 1AKE not available")


2025-12-16 17:54:03,094 - servers.structure_server - INFO - Splitting structure: output/1AKE.pdb


Test 3.2: Split 1AKE - select only chain A


                    INFO     Splitting structure: output/1AKE.pdb                          ]8;id=502228;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=542994;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1248\1248]8;;\

2025-12-16 17:54:03,095 - servers.structure_server - INFO - Inspecting molecules in: output/1AKE.pdb


                    INFO     Inspecting molecules in: output/1AKE.pdb                       ]8;id=305134;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=155670;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\

...

2025-12-16 17:54:03,131 - servers.structure_server - INFO - Wrote ligand: output/fadc5ba5/ligand_1.pdb


                    INFO     Wrote ligand: output/fadc5ba5/ligand_1.pdb                    ]8;id=75903;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=258160;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1454\1454]8;;\

2025-12-16 17:54:03,133 - servers.structure_server - INFO - Successfully split structure: 1 protein, 1 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 1 protein, 1 ligand, 0 ion, 0   ]8;id=948524;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=581103;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1481\1481]8;;\
                             water files                                                                           


 Split 1AKE (Chain A only)

✓ SUCCESS

Details:
  job_id: fadc5ba5
  output_dir: output/fadc5ba5
  source_file: output/1AKE.pdb
  file_format: pdb
  protein_files: ['output/fadc5ba5/protein_1.pdb']
  ligand_files: ['output/fadc5ba5/ligand_1.pdb']
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 2]
  include_types: ['protein', 'ligand', 'ion']

Extracted 1 protein chain(s)
Output directory: output/fadc5ba5


In [26]:
# Test 3.3: Split Boltz-2 predicted structure
print("Test 3.3: Split Boltz-2 predicted structure")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
result = split_molecules(boltz_cif)
show_result(result, "Split Boltz-2 Prediction")

if result['success']:
    print("\nProtein files:")
    for f in result['protein_files']:
        print(f"  - {f}")
    print("\nLigand files:")
    for f in result['ligand_files']:
        print(f"  - {f}")
    if result['ion_files']:
        print("\nIon files:")
        for f in result['ion_files']:
            print(f"  - {f}")
    
    # Show chain mapping
    print("\nChain to file mapping:")
    for info in result.get('chain_file_info', []):
        print(f"  Chain {info['chain_id']} ({info['chain_type']}): {info['file']}")


2025-12-16 17:54:03,138 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 3.3: Split Boltz-2 predicted structure


                    INFO     Splitting structure:                                          ]8;id=289208;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=97288;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1248\1248]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:54:03,139 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=924740;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=131986;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-16 17:54:03,212 - servers.structure_server - INFO - Wrote ligand: output/b2214f13/ligand_4.pdb


                    INFO     Wrote ligand: output/b2214f13/ligand_4.pdb                    ]8;id=823225;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=301876;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1454\1454]8;;\

2025-12-16 17:54:03,214 - servers.structure_server - INFO - Successfully split structure: 2 protein, 4 ligand, 0 ion, 0 water files


                    INFO     Successfully split structure: 2 protein, 4 ligand, 0 ion, 0   ]8;id=971362;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=698654;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1481\1481]8;;\
                             water files                                                                           


 Split Boltz-2 Prediction

✓ SUCCESS

Details:
  job_id: b2214f13
  output_dir: output/b2214f13
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  file_format: pdb
  protein_files: ['output/b2214f13/protein_1.pdb', 'output/b2214f13/protein_2.pdb']
  ligand_files: [complex data, 4]
  ion_files: []
  water_files: []
  all_chains: [complex data, 6]
  chain_file_info: [complex data, 6]
  include_types: ['protein', 'ligand', 'ion']

Protein files:
  - output/b2214f13/protein_1.pdb
  - output/b2214f13/protein_2.pdb

Ligand files:
  - output/b2214f13/ligand_1.pdb
  - output/b2214f13/ligand_2.pdb
  - output/b2214f13/ligand_3.pdb
  - output/b2214f13/ligand_4.pdb

Chain to file mapping:
  Chain A (protein): output/b2214f13/protein_1.pdb
  Chain B (protein): output/b2214f13/protein_2.pdb
  Chain C (ligand): output/b2214f13/ligand_1.pdb
  Chain D (ligand): output/b2214f13/ligand_2.pdb
  Chain E (ligand): output/b2214f13/ligand_3.pdb
  Chain F (ligand): output/b2214f13/lig

---
## Test 4: clean_protein

Test protein structure cleaning with PDBFixer.


In [27]:
# Test 4.1: Clean 1CRN (crambin - has disulfide bonds)
print("Test 4.1: Clean 1CRN (crambin with disulfide bonds)")

# First fetch and split
fetch_result = asyncio.run(fetch_molecules("1CRN", source="pdb"))
if fetch_result['success']:
    split_result = split_molecules(fetch_result['file_path'])
    if split_result['success'] and split_result['protein_files']:
        protein_pdb = split_result['protein_files'][0]
        
        result = clean_protein(protein_pdb)
        show_result(result, "Clean 1CRN")
        
        # Check disulfide bonds
        if result.get('disulfide_bonds'):
            print("\nDisulfide bonds detected:")
            for bond in result['disulfide_bonds']:
                print(f"  {bond['residue1']} <-> {bond['residue2']}")
    else:
        print("Failed to split 1CRN")
else:
    print("Failed to fetch 1CRN")


2025-12-16 17:54:03,221 - servers.structure_server - INFO - Fetching 1CRN from pdb


Test 4.1: Clean 1CRN (crambin with disulfide bonds)


                    INFO     Fetching 1CRN from pdb                                         ]8;id=915500;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=734297;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1CRN.pdb "HTTP/1.1   ]8;id=196656;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=356847;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-16 17:54:03,482 - servers.structure_server - INFO - Downloaded 1CRN to output/1CRN.pdb


...

2025-12-16 17:54:05,093 - servers.structure_server - INFO - pdb4amber conversion successful: output/7204a740/protein_1.amber.pdb


[12/16/25 17:54:05] INFO     pdb4amber conversion successful:                              ]8;id=173102;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=418118;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1883\1883]8;;\
                             output/7204a740/protein_1.amber.pdb                                                   

2025-12-16 17:54:05,095 - servers.structure_server - INFO - Successfully cleaned protein structure: output/7204a740/protein_1.amber.pdb


                    INFO     Successfully cleaned protein structure:                       ]8;id=402203;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=987422;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1918\1918]8;;\
                             output/7204a740/protein_1.amber.pdb                                                   


 Clean 1CRN

✓ SUCCESS

Details:
  output_file: output/7204a740/protein_1.amber.pdb
  input_file: output/7204a740/protein_1.pdb
  cap_termini_required: False
  statistics: {'initial_chains': 1, 'initial_residues': 46, 'final_residues': 46, 'final_atoms': 618}
  disulfide_bonds: [complex data, 3]
  pdbfixer_output: output/7204a740/protein_1.clean.pdb
  operations_summary: [complex data, dict]

Disulfide bonds detected:
  {'name': 'CYS', 'chain': 'A', 'index': 25} <-> {'name': 'CYS', 'chain': 'A', 'index': 15}
  {'name': 'CYS', 'chain': 'A', 'index': 31} <-> {'name': 'CYS', 'chain': 'A', 'index': 3}
  {'name': 'CYS', 'chain': 'A', 'index': 39} <-> {'name': 'CYS', 'chain': 'A', 'index': 2}


In [28]:
# Test 4.2: Clean with custom options
print("Test 4.2: Clean with custom options (with termini capping)")

if fetch_result['success'] and split_result['success']:
    protein_pdb = split_result['protein_files'][0]
    
    result = clean_protein(
        protein_pdb,
        cap_termini=True,
        ph=7.0
    )
    show_result(result, "Clean with Custom Options")
else:
    print("Skipped - previous test failed")


2025-12-16 17:54:05,101 - servers.structure_server - INFO - Cleaning protein structure: output/7204a740/protein_1.pdb


Test 4.2: Clean with custom options (with termini capping)


                    INFO     Cleaning protein structure: output/7204a740/protein_1.pdb     ]8;id=728537;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=609219;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1542\1542]8;;\

2025-12-16 17:54:05,102 - servers.structure_server - INFO - Loading structure with PDBFixer


                    INFO     Loading structure with PDBFixer                               ]8;id=210168;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=855400;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1571\1571]8;;\

...

2025-12-16 17:54:06,334 - servers.structure_server - INFO - pdb4amber conversion successful: output/7204a740/protein_1.amber.pdb


[12/16/25 17:54:06] INFO     pdb4amber conversion successful:                              ]8;id=999769;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=662432;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1883\1883]8;;\
                             output/7204a740/protein_1.amber.pdb                                                   

2025-12-16 17:54:06,336 - servers.structure_server - INFO - Successfully cleaned protein structure: output/7204a740/protein_1.amber.pdb


                    INFO     Successfully cleaned protein structure:                       ]8;id=157274;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=649619;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1918\1918]8;;\
                             output/7204a740/protein_1.amber.pdb                                                   


 Clean with Custom Options

✓ SUCCESS

Details:
  output_file: output/7204a740/protein_1.amber.pdb
  input_file: output/7204a740/protein_1.pdb
  cap_termini_required: True
  statistics: {'initial_chains': 1, 'initial_residues': 46, 'final_residues': 48, 'final_atoms': 627}
  disulfide_bonds: [complex data, 3]
  pdbfixer_output: output/7204a740/protein_1.clean.pdb
  operations_summary: [complex data, dict]


In [29]:
# Test 4.3: Clean non-existent file (error handling)
print("Test 4.3: Clean non-existent file")

result = clean_protein("/nonexistent/protein.pdb")
show_result(result, "Clean Non-existent File")

assert not result['success'], "Should fail for non-existent file"
print("\n✓ File not found error handling works")


2025-12-16 17:54:06,342 - servers.structure_server - INFO - Cleaning protein structure: /nonexistent/protein.pdb


Test 4.3: Clean non-existent file


                    INFO     Cleaning protein structure: /nonexistent/protein.pdb          ]8;id=219248;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=752047;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1542\1542]8;;\

2025-12-16 17:54:06,344 - servers.structure_server - ERROR - Input file not found: /nonexistent/protein.pdb


                    ERROR    Input file not found: /nonexistent/protein.pdb                ]8;id=258601;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=712489;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1561\1561]8;;\


 Clean Non-existent File

✗ FAILED

Errors:
  - Input file not found: /nonexistent/protein.pdb

Details:
  output_file: None
  input_file: /nonexistent/protein.pdb
  cap_termini_required: False
  statistics: {}
  disulfide_bonds: []

✓ File not found error handling works


---
## Test 5: clean_ligand

Test ligand cleaning using SMILES template matching.


In [30]:
# Test 5.1: Clean ligand from 1AKE (AP5A inhibitor)
print("Test 5.1: Clean ligand from 1AKE")

# Fetch and split 1AKE to get ligand
fetch_result = asyncio.run(fetch_molecules("1AKE", source="pdb"))
if fetch_result['success']:
    # Split to get ligand chains
    split_result = split_molecules(fetch_result['file_path'])
    
    if split_result['success'] and split_result['ligand_files']:
        ligand_pdb = split_result['ligand_files'][0]
        print(f"Ligand PDB: {ligand_pdb}")
        
        # Get ligand ID from chain info
        ligand_info = [c for c in split_result['chain_file_info'] if c['chain_type'] == 'ligand'][0]
        ligand_id = split_result['all_chains'][2]['residue_names']['unique_residues'][0]  # Get ligand name
        print(f"Ligand ID: {ligand_id}")
        
        # Clean ligand using SMILES template matching
        result = clean_ligand(
            ligand_pdb=ligand_pdb,
            ligand_id=ligand_id,  # AP5A
            target_ph=7.4,
            optimize=True
        )
        show_result(result, "Clean 1AKE Ligand (AP5A)")
        
        if result['success']:
            print(f"\nOutput SDF: {result['sdf_file']}")
            print(f"Net charge: {result['net_charge']}")
            print(f"SMILES source: {result['smiles_source']}")
    else:
        print("No ligand files found")
else:
    print("Failed to fetch 1AKE")


2025-12-16 17:54:06,350 - servers.structure_server - INFO - Fetching 1AKE from pdb


Test 5.1: Clean ligand from 1AKE


                    INFO     Fetching 1AKE from pdb                                         ]8;id=570499;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=277384;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#683\683]8;;\

                    INFO     HTTP Request: GET https://files.rcsb.org/download/1AKE.pdb "HTTP/1.1   ]8;id=675353;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=693336;file:///Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             200 OK"                                                                               

2025-12-16 17:54:06,706 - servers.structure_server - INFO - Downloaded 1AKE to output/1AKE.pdb


...

2025-12-16 17:54:08,486 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=469727;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=2407;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2314\2314]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-16 17:54:08,487 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=158034;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=179701;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2327\2327]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean 1AKE Ligand (AP5A)

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 57, Template atoms: 81, trying without H

Details:
  ligand_pdb: output/4b1337a4/ligand_1.pdb
  ligand_id: AP5
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4/ligand_1_prepared.sdf
  net_charge: -5
  charge_source: dimorphite
  mol_formal_charge: -5
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO[P@](=O)([O-])O[P@](=O)([O-])OP(=O)([O-])O[P@@](=O)([O-])O[P@@](=O)([O-])OC[C@H]2O[C@@H](n3cnc4c(N)ncnc43)[C@H](O)[C@@H]2O)[C@@H](O)[C@H]1O
  smiles_original: c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO[P@](=O)(O)O[P@](=O)(O)OP(=O)(O)O[P@@](=O)(O)O[P@@](=O)(O)OC[C@@H]4[C@H]([C@H]([C@@H](O4)n5cnc6c5ncnc6N)O)O)O)O)N
  smiles_source: ccd
  target_ph: 7.4
  num_atoms: 81
  num_heavy_atoms: 57
  optimized: True
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/4b1337a4

Output SDF: /Users/yasu/tmp/mcp-md/not

In [31]:
# Test 5.2: Clean SAH ligand from Boltz-2 prediction
print("Test 5.2: Clean SAH ligand from Boltz-2 prediction")

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Find SAH ligand chain
    sah_file = None
    sah_chain = None
    for info in split_result['chain_file_info']:
        if info['chain_type'] == 'ligand':
            # Get residue name from all_chains
            for chain in split_result['all_chains']:
                if chain['chain_id'] == info['chain_id']:
                    if 'SAH' in chain['residue_names']['unique_residues']:
                        sah_file = info['file']
                        sah_chain = chain
                        break
        if sah_file:
            break
    
    if sah_file:
        print(f"SAH ligand PDB: {sah_file}")
        
        result = clean_ligand(
            ligand_pdb=sah_file,
            ligand_id="SAH",
            target_ph=7.4,
            optimize=True
        )
        show_result(result, "Clean Boltz-2 SAH Ligand")
        
        if result['success']:
            print(f"\nOutput SDF: {result['sdf_file']}")
            print(f"Net charge: {result['net_charge']}")
    else:
        print("SAH ligand not found in Boltz-2 structure")
else:
    print("Failed to split Boltz-2 structure")


2025-12-16 17:54:08,493 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 5.2: Clean SAH ligand from Boltz-2 prediction


                    INFO     Splitting structure:                                          ]8;id=399858;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=967352;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1248\1248]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:54:08,494 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=620285;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=6340;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-16 17:54:09,719 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=502749;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=191133;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2314\2314]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-16 17:54:09,720 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=639206;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=459929;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2327\2327]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean Boltz-2 SAH Ligand

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 26, Template atoms: 45, trying without H

Details:
  ligand_pdb: output/b210efc7/ligand_1.pdb
  ligand_id: SAH
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_prepared.sdf
  net_charge: -1
  charge_source: dimorphite
  mol_formal_charge: -1
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)[O-])[C@@H](O)[C@H]1O
  smiles_original: c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CSCC[C@@H](C(=O)O)N)O)O)N
  smiles_source: ccd
  target_ph: 7.4
  num_atoms: 45
  num_heavy_atoms: 26
  optimized: True
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7

Output SDF: /Users/yasu/tmp/mcp-md/notebooks/output/b210efc7/ligand_1_prepared.sdf
Net charge: -1


In [32]:
# Test 5.3: Clean ligand with user-provided SMILES
print("Test 5.3: Clean ligand with user-provided SMILES")

# Use Boltz-2 SAH ligand with explicit SMILES
boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Get first ligand file
    ligand_file = split_result['ligand_files'][0]
    ligand_chain = split_result['chain_file_info'][2]  # First ligand
    ligand_name = split_result['all_chains'][2]['residue_names']['unique_residues'][0]
    
    print(f"Ligand: {ligand_name}")
    print(f"File: {ligand_file}")
    
    # SAH SMILES from PDB CCD
    sah_smiles = "Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O"
    
    result = clean_ligand(
        ligand_pdb=ligand_file,
        ligand_id=ligand_name,
        smiles=sah_smiles,  # User-provided SMILES
        target_ph=7.4,
        optimize=False  # Skip optimization for speed
    )
    show_result(result, "Clean Ligand with User SMILES")
    
    if result['success']:
        print(f"\nSMILES source: {result['smiles_source']}")  # Should be 'user'
else:
    print("Failed to get ligand from Boltz-2")


2025-12-16 17:54:09,726 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 5.3: Clean ligand with user-provided SMILES


                    INFO     Splitting structure:                                          ]8;id=176843;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=389242;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1248\1248]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:54:09,728 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=827621;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=884498;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-16 17:54:09,836 - servers.structure_server - INFO - Wrote prepared ligand: /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2/ligand_1_prepared.sdf


                    INFO     Wrote prepared ligand:                                        ]8;id=117219;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=969221;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2314\2314]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-16 17:54:09,837 - servers.structure_server - INFO - Successfully cleaned ligand: /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2/ligand_1_prepared.sdf


                    INFO     Successfully cleaned ligand:                                  ]8;id=485135;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=853019;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2327\2327]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2/ligand_1_pre                         
                             pared.sdf                                                                             


 Clean Ligand with User SMILES

✓ SUCCESS

Warnings:
  - Template matching with H failed: Template matching failed: No matching found. PDB atoms: 26, Template atoms: 45, trying without H

Details:
  ligand_pdb: output/1cd8f7a2/ligand_1.pdb
  ligand_id: SAH
  sdf_file: /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2/ligand_1_prepared.sdf
  net_charge: -1
  charge_source: dimorphite
  mol_formal_charge: -1
  smiles_used: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)[O-])[C@@H](O)[C@H]1O
  smiles_original: Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O
  smiles_source: user
  target_ph: 7.4
  num_atoms: 45
  num_heavy_atoms: 26
  optimized: False
  optimization_converged: False
  output_dir: /Users/yasu/tmp/mcp-md/notebooks/output/1cd8f7a2

SMILES source: user


---
## Test 6: run_antechamber_robust

Test GAFF2 parameterization with AM1-BCC charges.


In [33]:
# Test 6.1: Run antechamber on cleaned SAH ligand
print("Test 6.1: Run antechamber on SAH ligand (GAFF2 + AM1-BCC)")

# First clean the SAH ligand
boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"
split_result = split_molecules(boltz_cif)

if split_result['success'] and split_result['ligand_files']:
    # Find SAH ligand
    sah_file = None
    for info in split_result['chain_file_info']:
        if info['chain_type'] == 'ligand':
            for chain in split_result['all_chains']:
                if chain['chain_id'] == info['chain_id'] and 'SAH' in chain['residue_names']['unique_residues']:
                    sah_file = info['file']
                    break
        if sah_file:
            break
    
    if sah_file:
        # Clean ligand first
        clean_result = clean_ligand(
            ligand_pdb=sah_file,
            ligand_id="SAH",
            target_ph=7.4
        )
        
        if clean_result['success']:
            print(f"Cleaned SDF: {clean_result['sdf_file']}")
            print(f"Net charge: {clean_result['net_charge']}")
            
            # Run antechamber
            result = run_antechamber_robust(
                ligand_file=clean_result['sdf_file'],
                net_charge=clean_result['net_charge'],
                residue_name="SAH"
            )
            show_result(result, "Antechamber SAH")
            
            if result['success']:
                print(f"\nGenerated files:")
                print(f"  MOL2: {result['mol2']}")
                print(f"  FRCMOD: {result['frcmod']}")
                print(f"  Total charge: {result['total_charge']:.4f}")
                
                # Check frcmod validation
                if result['frcmod_validation']:
                    if result['frcmod_validation']['valid']:
                        print("  frcmod: ✓ Valid")
                    else:
                        print(f"  frcmod: ✗ {result['frcmod_validation']['attn_count']} parameters need attention")
        else:
            print(f"Clean failed: {clean_result['errors']}")
    else:
        print("SAH not found")
else:
    print("Failed to split structure")


2025-12-16 17:54:09,842 - servers.structure_server - INFO - Splitting structure: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 6.1: Run antechamber on SAH ligand (GAFF2 + AM1-BCC)


                    INFO     Splitting structure:                                          ]8;id=498280;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=239573;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#1248\1248]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:54:09,844 - servers.structure_server - INFO - Inspecting molecules in: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


                    INFO     Inspecting molecules in:                                       ]8;id=205086;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=647812;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#905\905]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

...

2025-12-16 17:54:50,202 - servers.structure_server - INFO - Successfully generated PDB: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.amber.pdb


                    INFO     Successfully generated PDB:                                   ]8;id=518388;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=192450;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2603\2603]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_pre                         
                             pared.amber.pdb                                                                       

2025-12-16 17:54:50,205 - servers.structure_server - INFO - Successfully parameterized ligand: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.gaff.mol2


                    INFO     Successfully parameterized ligand:                            ]8;id=514736;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=833138;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2645\2645]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_pre                         
                             pared.gaff.mol2                                                                       


 Antechamber SAH

✓ SUCCESS

Details:
  mol2: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.gaff.mol2
  frcmod: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.frcmod
  pdb: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.amber.pdb
  charge_used: -1
  charge_method: bcc
  atom_type: gaff2
  residue_name: SAH
  charges: [complex data, 45]
  total_charge: -1.0000000000000016
  frcmod_validation: [complex data, dict]
  sqm_diagnostics: None
  charge_estimation: None
  diagnostics_dir: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/diagnostics

Generated files:
  MOL2: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.gaff.mol2
  FRCMOD: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.frcmod
  Total charge: -1.0000
  frcmod: ✓ Valid


In [34]:
# Test 6.2: Run antechamber with auto charge estimation
print("Test 6.2: Run antechamber with auto charge estimation")

# Use the same cleaned SDF but let antechamber estimate the charge
if 'clean_result' in dir() and clean_result['success']:
    result = run_antechamber_robust(
        ligand_file=clean_result['sdf_file'],
        net_charge=None,  # Auto-estimate
        residue_name="SAH",
        charge_method="bcc",
        atom_type="gaff2"
    )
    show_result(result, "Antechamber (Auto Charge)")
    
    if result['success']:
        print(f"\nCharge estimation:")
        if result['charge_estimation']:
            print(f"  Estimated: {result['charge_estimation'].get('estimated_charge_at_ph')}")
            print(f"  Confidence: {result['charge_estimation'].get('confidence')}")
        print(f"  Charge used: {result['charge_used']}")
else:
    print("Skipped - clean_result not available")


2025-12-16 17:54:50,212 - servers.structure_server - INFO - Running robust antechamber: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.sdf


Test 6.2: Run antechamber with auto charge estimation


                    INFO     Running robust antechamber:                                   ]8;id=719071;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=751061;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2385\2385]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_pre                         
                             pared.sdf                                                                             

2025-12-16 17:54:50,213 - servers.structure_server - INFO - Auto-estimating net charge...


                    INFO     Auto-estimating net charge...                                 ]8;id=425399;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=613339;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2427\2427]8;;\

...

2025-12-16 17:55:24,068 - servers.structure_server - INFO - Successfully generated PDB: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.amber.pdb


                    INFO     Successfully generated PDB:                                   ]8;id=658123;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=541356;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2603\2603]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_pre                         
                             pared.amber.pdb                                                                       

2025-12-16 17:55:24,071 - servers.structure_server - INFO - Successfully parameterized ligand: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.gaff.mol2


                    INFO     Successfully parameterized ligand:                            ]8;id=463507;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=599762;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2645\2645]8;;\
                             /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_pre                         
                             pared.gaff.mol2                                                                       


 Antechamber (Auto Charge)

✓ SUCCESS

Details:
  mol2: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.gaff.mol2
  frcmod: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.frcmod
  pdb: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/ligand_1_prepared.amber.pdb
  charge_used: -1
  charge_method: bcc
  atom_type: gaff2
  residue_name: SAH
  charges: [complex data, 45]
  total_charge: -1.0000000000000016
  frcmod_validation: [complex data, dict]
  sqm_diagnostics: None
  charge_estimation: [complex data, dict]
  diagnostics_dir: /Users/yasu/tmp/mcp-md/notebooks/output/2807a93e/diagnostics

Charge estimation:
  Estimated: -1
  Confidence: high
  Charge used: -1


---
## Test 7: Integration Test

Test complete workflows: fetch -> split -> clean_protein + clean_ligand -> antechamber


In [35]:
# Test 7.1: Complete workflow using prepare_complex
print("Test 7.1: Complete Boltz-2 workflow using prepare_complex")
print("="*60)

boltz_cif = "boltz_results_ligand/predictions/ligand/ligand_model_0.cif"

# Run complete workflow with a single function call
result = prepare_complex(
    structure_file=boltz_cif,
    ph=7.4,
    cap_termini=False,
    process_proteins=True,
    process_ligands=True,
    run_parameterization=True,
    optimize_ligands=True,
    # Optional: provide SMILES for specific ligands
    # ligand_smiles={"SAH": "Nc1ncnc2c1ncn2[C@@H]1O[C@H](CSCC[C@H](N)C(=O)O)[C@@H](O)[C@H]1O"}
)

show_result(result, "prepare_complex Result")

if result['success']:
    print("\n--- Summary ---")
    print(f"Output directory: {result['output_dir']}")
    
    # Show inspection summary
    if result['inspection']:
        summary = result['inspection'].get('summary', {})
        print(f"\nStructure: {summary.get('num_protein_chains', 0)} proteins, "
              f"{summary.get('num_ligand_chains', 0)} ligands")
    
    # Show processed proteins
    print(f"\nProteins processed: {len(result['proteins'])}")
    for p in result['proteins']:
        status = "✓" if p['success'] else "✗"
        print(f"  {status} Chain {p['chain_id']}: {p.get('output_file', 'N/A')}")
        if p['success'] and p.get('statistics'):
            print(f"      Atoms: {p['statistics'].get('final_atoms', 'N/A')}")
    
    # Show processed ligands
    print(f"\nLigands processed: {len(result['ligands'])}")
    for l in result['ligands']:
        status = "✓" if l['success'] else "✗"
        print(f"  {status} {l['ligand_id']} (Chain {l['chain_id']})")
        if l['success']:
            print(f"      SDF: {l.get('sdf_file', 'N/A')}")
            print(f"      MOL2: {l.get('mol2_file', 'N/A')}")
            print(f"      Charge: {l.get('net_charge', 'N/A')}")

print("\n" + "="*60)
print("Workflow complete!")


2025-12-16 17:55:24,079 - servers.structure_server - INFO - Preparing complex: boltz_results_ligand/predictions/ligand/ligand_model_0.cif


Test 7.1: Complete Boltz-2 workflow using prepare_complex


                    INFO     Preparing complex:                                            ]8;id=969892;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=495809;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2947\2947]8;;\
                             boltz_results_ligand/predictions/ligand/ligand_model_0.cif                            

2025-12-16 17:55:24,081 - servers.structure_server - INFO - Step 1: Inspecting structure...


                    INFO     Step 1: Inspecting structure...                               ]8;id=764579;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=778341;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#2981\2981]8;;\

...

2025-12-16 17:56:34,489 - servers.structure_server - INFO -   Ligands processed: 2/4


                    INFO       Ligands processed: 2/4                                      ]8;id=926256;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=605055;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#3260\3260]8;;\

2025-12-16 17:56:34,490 - servers.structure_server - INFO -   Merged PDB: output/fe6ce451/merged.pdb


                    INFO       Merged PDB: output/fe6ce451/merged.pdb                      ]8;id=590354;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py\structure_server.py]8;;\:]8;id=1854;file:///Users/yasu/tmp/mcp-md/notebooks/../servers/structure_server.py#3262\3262]8;;\


 prepare_complex Result

✓ SUCCESS

Warnings:
  - Ligand LIG1 cleaning failed: ['No SMILES found for ligand LIG1', "Hint: Provide SMILES manually via the 'smiles' parameter, or add it to KNOWN_LIGAND_SMILES dictionary"]
  - Ligand LIG1 cleaning failed: ['No SMILES found for ligand LIG1', "Hint: Provide SMILES manually via the 'smiles' parameter, or add it to KNOWN_LIGAND_SMILES dictionary"]

Details:
  job_id: d999b4d3
  output_dir: output/66ea07a4
  source_file: boltz_results_ligand/predictions/ligand/ligand_model_0.cif
  inspection: [complex data, dict]
  split: [complex data, dict]
  proteins: [complex data, 2]
  ligands: [complex data, 4]
  merged_pdb: output/fe6ce451/merged.pdb
  merge_result: [complex data, dict]

--- Summary ---
Output directory: output/66ea07a4

Structure: 2 proteins, 4 ligands

Proteins processed: 2
  ✓ Chain A: output/66ea07a4/protein_1.amber.pdb
      Atoms: 5848
  ✓ Chain B: output/66ea07a4/protein_2.amber.pdb
      Atoms: 5848

Ligands processed: 4
  ✓ SA

---
## Summary

This notebook tested all tools in `structure_server.py`:

| Tool | Tests | Purpose |
|------|-------|---------|
| `fetch_molecules` | 2 | Download structures from PDB |
| `inspect_molecules` | 2 | Inspect structure files and analyze chains/molecules |
| `split_molecules` | 3 | Split multi-chain structures (including Boltz-2) |
| `clean_protein` | 3 | Clean and prepare proteins for MD |
| `clean_ligand` | 3 | Clean ligands using SMILES template matching |
| `run_antechamber_robust` | 2 | GAFF2 parameterization with AM1-BCC charges |
| `prepare_complex` | 1 | Complete workflow (split + clean + parameterize) |

### Key Features Tested:
- **LLM-friendly error handling**: All tools return structured `success`/`errors`/`warnings` fields
- **SMILES template matching**: Correct bond orders from CCD or user-provided SMILES
- **pH-dependent protonation**: Dimorphite-DL for correct protonation state
- **GAFF2 parameterization**: AM1-BCC charges with robust error handling
- **frcmod validation**: Check for missing/estimated parameters
- **Boltz-2 support**: Full workflow for AI-predicted protein-ligand complexes
- **One-step workflow**: `prepare_complex` combines all steps for convenience


In [36]:
print("All tests completed!")


All tests completed!
